<a href="https://colab.research.google.com/github/animesharma3/Text-Summarization-using-T5-transformers-and-Pytorch-Lightning/blob/main/Text_Summarization_Using_Transformer_T5_and_Pytorch_Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine-tuning LLama 2 model**

This notebook was ran on Colab. We use Meta AI's open-source pretrained llama 2 model available on huggingface.

base_model = 'meta-llama/Llama-2-7b-hf'

Referneces:
tutorial: https://www.youtube.com/watch?v=MDA3LUKNl1E

github: https://github.com/curiousily/Get-Things-Done-with-Prompt-Engineering-and-LangChain/blob/master/14.fine-tuning-llama-2-7b-on-custom-dataset.ipynb


tutorial: https://www.kaggle.com/code/mahimairaja/fine-tuning-llama-2-tweet-summarization


**Install required libraries**

In [ ]:
!pip install --quiet transformers
!pip install --quiet pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 11.2 MB/s eta 0:00:00


In [ ]:
!pip install --quiet transformers
!pip install --quiet pytorch-lightning
!pip install torchtext==0.6.0
!pip install -qqq peft==0.5.0 --progress-bar off
!pip install -qqq trl==0.7.1 --progress-bar off
!pip install bitsandbytes-cuda110 bitsandbytes
!pip install accelerate
!pip install -i https://test.pypi.org/simple/ bitsandbytes
!pip install torchtext==0.6.0
!pip install -qqq peft==0.5.0 --progress-bar off
!pip install -qqq trl==0.7.1 --progress-bar off

## Restart runtime session before proceeding

**Import libraries**

In [ ]:
import json
import re
from pprint import pprint

import pandas as pd
import numpy as np
import torch
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
torch.cuda.empty_cache()
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap
import transformers
from transformers import (
    AutoModelForCausalLM,
    TrainingArguments,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from trl import SFTTrainer
from tqdm.auto import tqdm
DEVICE = "cude:0" if torch.cuda.is_available() else 'cpu'
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10
pl.seed_everything(42)


**If running on colab, please mount and navigate to the desired directory**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
cd "/content/drive/My Drive/ML project"

Mounted at /content/drive


**Specify the LLM model to import from huggingface in MODEL_NAME**

In [ ]:
MODLE_NAME = 'meta-llama/Llama-2-7b-hf'

**Dataset**
- Here we import the dataset.
- Cleaning was done as described in the report and presentation. The cleaning is in the following cells
- We perform based preparation

In [ ]:
df = pd.read_excel("scientific_papers_pubmed_test.xlsx")
df = df.head(2000) ##take only 2000 articles due to memory issues
df

In [ ]:
print('Dataset before dropping null : ', df.shape)
df.columns = ['article', 'summary']
df = df.dropna()
print('Dataset after dropping null : ', df.shape)

Dataset before dropping null :  (2000, 2)
Dataset after dropping null :  (2000, 2)


In [ ]:
df.rename(columns={'article': 'text', 'abstract':'summary'}, inplace = True)
df.head()

,text,summary
0,anxiety affects quality of life in those livin...,research on the implications of anxiety in pa...
1,small non - coding rnas are transcribed into m...,"small non - coding rnas include sirna , mirna..."
2,ohss is a serious complication of ovulation in...,objective : to evaluate the efficacy and safe...
3,congenital adrenal hyperplasia ( cah ) refers ...,congenital adrenal hyperplasia is a group of ...
4,type 1 diabetes ( t1d ) results from the destr...,objective(s):pentoxifylline is an immunomodul...


**Specify the prompt structure to follow**

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
Below is a scientific paper from pubmed. Write a summary.
""".strip()


def generate_training_prompt(
      text: str, summary: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
  ) -> str:
      text_chunk = text[:1024]  # Extracting the first 1024 characters of the text

      summary_chunk = summary[:512]  # Extracting the first 512 characters of the summary

      return f"""### Instruction: {system_prompt}

  ### Input:
  {text_chunk.strip()}

  ### Response:
  {summary_chunk}
  """.strip()

**Data cleaning as per our description**

In [ ]:
import string
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@[^\s]+", "", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[()]", " ", text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return re.sub(r"\^[^ ]+", "", text)


def generate_text(row):
    summary = clean_text(row['summary'])
    text = clean_text(row['text'])

    return {
        "paper": text,
        "summary": summary,
        "text": generate_training_prompt(text, summary),
    }


**Split the dataset and prepare the dataset to be saved**

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.1)
train_df.shape, test_df.shape, val_df.shape

((1440, 2), (400, 2), (160, 2))

In [ ]:
def process_dataset(data: Dataset) -> None:
    """
    This func remove all cols excepts conversation, summary and text
    """
    return (
        data.shuffle(seed=42)
        .map(generate_text)
        .remove_columns(
            ['__index_level_0__']
        )
    )

In [ ]:
from datasets import Dataset, DatasetDict
# dataset = Dataset()
dataset_train = process_dataset(Dataset.from_pandas(train_df))
dataset_validation = process_dataset(Dataset.from_pandas(val_df))
dataset_test = process_dataset(Dataset.from_pandas(test_df))




Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
dataset = DatasetDict({
    "train": dataset_train,
    "validation": dataset_validation,
    "test": dataset_test,
})

In [ ]:
dataset.save_to_disk("experiments-llama/dataset.hf")

Saving the dataset (0/1 shards):   0%|          | 0/1440 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/160 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/400 [00:00<?, ? examples/s]

**Login to huggingface (Important to load the model):**

In [ ]:
notebook_login()
## you will need to have a huggingface token

**Load the model**

In [ ]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODLE_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto"
    )

    tokenizer = AutoTokenizer.from_pretrained(MODLE_NAME,use_auth_token=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [ ]:
import accelerate
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

**Model Quantation configs**

In [ ]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16'}

**Specify PEFT Configurations**

In [ ]:
lora_alpha = 32
lora_dropout = 0.05
lora_r = 16

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

**Specify the output directory**

In [ ]:
OUTPUT_DIR = "experiments-llama/v2"


**Specify training arguments**

In [ ]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=2, ## impacts memeory allocatoin
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)


In [ ]:
ttrainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=3000,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

In [ ]:
torch.cuda.empty_cache()

ttrainer.train()


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
72,1.698000,1.673526
144,1.456100,1.622305
216,1.882000,1.609851
288,1.640000,1.606107
360,2.163400,1.604342


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=360, training_loss=1.6755385051170986, metrics={'train_runtime': 5721.2465, 'train_samples_per_second': 0.503, 'train_steps_per_second': 0.063, 'total_flos': 4.712065861715558e+16, 'train_loss': 1.6755385051170986, 'epoch': 2.0})

In [ ]:
kwargs = {
    "dataset_tags": "pubmed-dataset",
    "finetuned_from": "meta-llama/Llama-2-7b-hf",
    "tasks": "text-generation",
}

In [ ]:
ttrainer.push_to_hub(**kwargs)

In [ ]:
ttrainer.save_model()


In [ ]:
ttrainer.model


In [ ]:
from peft import AutoPeftModelForCausalLM

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    low_cpu_mem_usage=True,
)

merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model", safe_serialization=True)
tokenizer.save_pretrained("merged_model")

In [ ]:
new_model = 'finetuned-llama-2-v2'
merged_model.push_to_hub(new_model, max_shard_size='2GB')
tokenizer.push_to_hub(new_model)